# Implementação do DSA com o *SageMath*

## Descrição do Exercício

Construir uma classe Python que implemente o DSA. A implementação deve:
  1. A iniciação,  receber como parâmetros o tamanho  dos primos $p$ e $q$.
  2. Conter funções para assinar digitalmente e verificar a assinatura.

## Descrição da Implementação

Para a inicialização do DSA existiu a criação duma classe que tivesse como parâmetros de construção os tamanhos dos primos $p$ e $q$, também comummente chamados de $L$ e $N$, com valores recomendados de $(1024, 160), (2048, 224), (2048, 256), (3072, 256)$.

### Geração dos parâmetros DSA

$q$ é um **número primo** de tamanho $2^{N–1} < p < 2^{N}$;

$p$ é um **número primo** de tamanho $2^{L–1} < p < 2^{L}$, tal que $(p-1)$ tem de ser múltiplo de $q$;

$g$ é um gerador tal que $g := h^{(p - 1)/q} \mod p$.

### Geração das chaves DSA

Após existirem os parâmetros DSA $(p, q, g)$ podemos assim criar as chaves DSA da seguinte forma:

- Seleção dum inteiro que $\in \{ 1 \ldots q-1 \}$ que servirá de **chave privada DSA*;
- A geração da **chave pública** utilizando a seguinte equação $g^x \mod p$

### Assinatura DSA

Agora com as chaves DSA já geradas, podemos efetuar uma assinatura duma mensagem simples, usando para tal efeito a **chave privada** criada anteriormente.

- Seleção dum inteiro $k$ que $\in \{ 1 \ldots q-1 \}$;
- A geração do $r$ sendo que $r := (g^{k}\bmod\,p)\bmod\,q$;
- A geração do $s$ sendo que $s := (k^{-1}(H(m)+xr))\bmod\,q$;

Ficando assim com o par que representa a assinatura DSA $(r,s)$.

### Verificação DSA

Para verificar que a assinatura DSA $(r,s)$ é válida para a mensagem $m$ segue-se os seguintes passos:
* Verifica-se se $0 < r < q$ e $0 < s < q$;
* Gera-se $ w := s^{-1} \bmod\,q$.
* Gera-se $u_1 := H(m) \cdot w\, \bmod\,q$.
* Gera-se $u_2 := r \cdot w\, \bmod\,q$.
* Gera-se $  v := \left(g^{u_1}y^{u_2} \bmod\,p\right) \bmod\,q$.

A assinatura é válida para a mensagem se e só se $v == r$.

## Resolução do Exercício

In [1]:
from random import randint


class DSA:
    
    def __init__(self, l, n):
        
        self.tamprimop = l
        if n <= 256:
            self.tamprimoq = n
        else: self.tamprimoq = 256 # Por causa do tamanho de N <= | H | e a hash a usar é SHA-256
            
    
    def gerarparametros(self):
 
        # Escolha de primos grandes
        
        self.p = random_prime ( 2^self.tamprimop , proof=False , lbound=2^(self.tamprimop-1) )
        print "P: ",self.p
        
        self.q = random_prime ( 2^self.tamprimoq , proof=False , lbound=2^(self.tamprimoq-1) )
        
        print "Q: ",self.q

        h = Integer(1, self.p-1)
            
        self.g = ( ( h ^ ( (self.p-1) / self.q ) ) % self.p)
            
    def gerarchaves(self):

        privkey = randint(1, self.q-1) # privkey é um inteiro de {1 .. q-1}

        pubkey = power_mod(self.g, privkey, self.p) # pubkey = g^x mod p

        return (privkey, pubkey)
    
    def assinar(self, privkey, mensagem):
        
        print "--- A assinar a mensagem: {} ---".format(mensagem)
        
        # Criação do inteiro random k
        k = randint(1,self.q-1)
        
        # Criar o par da assinatura DSA
        r = ( ( self.g^k ) % self.p ) % self.q
        s = ( ( k^-1 ) * ( hash ( mensagem ) + privkey * r ) ) % self.q
        
        # Assinatura = ( r , s ) 
        return (r,s) 
    
    
    def verificar(self, assinatura, pubkey, mensagem):
        
        print "--- A verificar a assinatura para a mensagem: {} ---".format(mensagem)
        
        r = assinatura[0]
        s = assinatura[1]
        
        w = (s^-1) % self.q
        
        u1 = (hash (mensagem) * w) % self.q
        u2 = (r * w) % self.q
        
        v = ( ( (self.g^u1) * (pubkey^u2) ) % self.p ) % self.q
        
        if v == r: print "--- Assinatura verificada para a mensagem. ---"
        else: print "--- Assinatura não corresponde à mensagem. ---"

## Teste da Classe

Segue-se abaixo um teste desenvolvido para esta classe. **A ideia passa pelos seguintes passos:**

- Gerar um par de chaves (privkey,pubkey)
- Assinar essa mensagem inicial com a *Private Key* X
- Criar uma nova mensagem diferente.
- Verificar a assinatura com essa nova mensagem

In [2]:
# Criar uma instância e sua inicialização
myDSA = DSA(2048, 256)

# Gerar os parâmetros de instância (p, q, g)
myDSA.gerarparametros()

# Obter o par de chaves DSA
(privkey, pubkey) = myDSA.gerarchaves()

# Assinar uma pequena mensagem
assinatura =  myDSA.assinar(privkey, "Pequena mensagem")

# Verificar se a assinatura é da mensagem
myDSA.verificar(assinatura, pubkey, "Pequena Mensagem")

P:  26755791663670275100506220825769201629927866604035063846376967996016178449525930272746012293573608009397531757246610690510208505447733347624902951108382558191063953656539897874235799679545212894190981364911341619558403864593635171673453195249460659052770466380929332949016663623604122859327419821460233902381576447746901156117410270625018544498469745990952829785041388385780577688688942016700134639380379443403017150538013734166352559191142616550925539923420576886726957648475882329130004225048308597609710019381123593032537727707919891366097394669891172455226294637238312135770574138974904048155317364392591096573749
Q:  104433067033800923149449905571372533070605156566956955797894614612183632462729
--- A assinar a mensagem: Pequena mensagem ---
--- A verificar a assinatura para a mensagem: Pequena Mensagem ---
--- Assinatura verificada para a mensagem. ---


## Observações Finais

- O algoritmo **DSA** encontra-se detalhadamente descrito em várias fontes *online*
- Depois de compreender o seu funcionamento torna-se mais simples de começar a aplicar toda a ideia em SageMath

## Referências

- Wikipedia, SHA-2 https://en.wikipedia.org/wiki/SHA-2 (Acedido a 14 de Abril 2020)
- Python, HashLib package https://docs.python.org/3/library/hashlib.html (Acedido a 14 de Abril 2020)
- Wikipedia, Digital Signature Algorithm https://en.wikipedia.org/wiki/Digital_Signature_Algorithm (Acedido a 14 abril 2020)